# 梯度下降和随机梯度下降
梯度下降（Gradient Descent）和随机梯度下降（Stochastic Gradient Descent）都是用于机器学习和深度学习中的优化算法，用于调整模型参数以最小化损失函数。它们之间的主要区别在于优化过程的方式和效率。

1. 梯度下降（Gradient Descent）：
   - 在梯度下降中，每一轮迭代都会使用整个训练数据集来计算损失函数的梯度（或导数），然后通过梯度来更新模型参数。
   - 优点：通常能够更稳定地朝向全局最小值收敛，特别适合在小数据集上使用。
   - 缺点：计算整个数据集的梯度可能会很慢，特别是在大规模数据集上，因为它需要大量的计算资源。

2. 随机梯度下降（Stochastic Gradient Descent，SGD）：
   - 在随机梯度下降中，每一轮迭代都随机选择一个样本来计算损失函数的梯度，然后使用这个梯度来更新模型参数。
   - 优点：由于随机性，SGD通常更快地进行更新，特别适合大规模数据集。此外，SGD可以跳出局部极小值。
   - 缺点：由于随机性，它在迭代过程中的损失函数值波动较大，不如梯度下降稳定。另外，由于每次迭代只使用一个样本，可能需要更多迭代次数才能收敛到最小值。

在实际应用中，还有一些中间方法，如小批量随机梯度下降（Mini-batch Gradient Descent），它结合了梯度下降和随机梯度下降的优点，使用一小批数据来计算梯度。选择哪种方法取决于数据集的大小、计算资源和模型的特性。

In [1]:
# 使用小批量随机梯度下降算法结合pytorch写一个实例
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 步骤1: 准备数据集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)


5.3%


Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data\cifar-10-python.tar.gz


ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。

In [ ]:
# 步骤2: 定义神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6 * 13 * 13, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 6 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
# 步骤3: 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 存储损失值的列表
loss_values = []

In [ ]:
# 步骤4: 训练模型
for epoch in range(5):  # 5个轮次
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 步骤4.1: 梯度清零
        optimizer.zero_grad()

        # 步骤4.2: 前向传播
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # 步骤4.3: 反向传播
        loss.backward()

        # 步骤4.4: 参数更新
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # 每200个小批量打印一次损失
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 200}')
            loss_values.append(running_loss / 200)
            running_loss = 0.0

In [ ]:
# 步骤5: 损失值可视化
plt.plot(loss_values)
plt.xlabel('Training Iterations')
plt.ylabel('Loss')
plt.title('Training Loss over Iterations')
plt.show()

In [ ]:
# 步骤6: 模型测试
correct = 0
total = 0
with torch.no_grad():  # 在测试时不需要计算梯度
    for data in testloader:  # 使用测试集数据
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


# 代码分析
上述代码定义了一个神经网络模型，通常用于处理CIFAR-10图像分类任务。以下是对该代码的逐句解释：

1. `class Net(nn.Module):`：这一行声明了一个名为 `Net` 的类，它是一个PyTorch神经网络模型的定义。

2. `def __init__(self):`：这是 `Net` 类的构造函数，在初始化模型时被调用。

3. `super(Net, self).__init__():`：这一行调用父类 `nn.Module` 的构造函数，确保正确初始化模型。

4. `self.conv1 = nn.Conv2d(3, 6, 5)`：这一行定义了一个卷积层 (`nn.Conv2d`)，该层有3个输入通道（RGB图像），6个输出通道（卷积核数量），卷积核的大小为5x5。

5. `self.pool = nn.MaxPool2d(2, 2)`：这一行定义了一个最大池化层 (`nn.MaxPool2d`)，池化核大小为2x2，步幅也为2。

6. `self.fc1 = nn.Linear(6 * 13 * 13, 120)`：这一行定义了一个全连接层 (`nn.Linear`)，它将前一层的输出（6 * 13 * 13维度）连接到一个具有120个神经元的全连接层。

7. `self.fc2 = nn.Linear(120, 84)`：这一行定义了另一个全连接层，将120维输入连接到84维输出。

8. `self.fc3 = nn.Linear(84, 10)`：这一行定义了最后一个全连接层，将84维输入连接到10维输出。在CIFAR-10中，有10个不同的类别。

9. `def forward(self, x):`：这是模型的前向传播函数，它定义了数据在网络中如何传递。

10. `x = self.pool(F.relu(self.conv1(x)))`：这一行首先将输入 `x` 传递给第一个卷积层 `conv1`，然后应用ReLU激活函数，最后通过最大池化层 `pool` 进行下采样。

11. `x = x.view(-1, 6 * 13 * 13)`：在这里，输出张量 `x` 被展平为一维，以便输入到全连接层。

12. `x = F.relu(self.fc1(x))`：接着，数据通过第一个全连接层 `fc1`，并再次应用ReLU激活函数。

13. `x = F.relu(self.fc2(x))`：然后，数据通过第二个全连接层 `fc2`，再次应用ReLU激活函数。

14. `x = self.fc3(x)`：最后，数据通过最后一个全连接层 `fc3`，得到输出，这是一个10维的向量，对应于CIFAR-10中的10个类别。

15. `return x`：前向传播函数返回最终的输出，通常用于计算损失并进行反向传播更新模型参数。